<a href="https://colab.research.google.com/github/anudeep22003/nlp-training/blob/main/(chapter_9)_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk import word_tokenize, re
import matplotlib.pyplot as plt
from pprint import pprint
nltk.download('brown')
nltk.download('treebank')
nltk.download('words')


from nltk.corpus import conll2000
nltk.download('conll2000')

import math 
import random

from nltk.corpus import treebank
nltk.download('treebank')

from collections import defaultdict

nltk.download('ppattach')


nltk.download('book_grammars')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package ppattach to /root/nltk_data...
[nltk_data]   Unzipping corpora/ppattach.zip.
[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/book_grammars.zip.


True

# Exercise 1 

☼ What constraints are required to correctly parse word sequences like I am happy and she is happy but not *you is happy or *they am happy? Implement two solutions for the present tense paradigm of the verb be in English, first taking Grammar (6) as your starting point, and then taking Grammar (18) as the starting point.

In [ ]:
sentc_1 = "I am happy".split()
sentc_2 = "she is happy".split()

sentw_1 = "you is happy".split()
sentw_2 = "they am happy".split()

grammar = nltk.CFG.fromstring("""

  S -> NP_1_SG VP_1_SG      
  S -> NP_3_SG VP_3_SG      
  S -> NP_3_PL VP_3_PL      
  S -> NP_2_SG VP_2_SG      

  NP_1_SG -> RB_1_SG 
  VP_1_SG -> V_1_SG N                                                                                     

  NP_3_SG -> RB_3_SG
  VP_3_SG -> V_3_SG N

  NP_2_SG -> RB_2_SG 
  VP_2_SG -> V_2_SG N

  NP_3_PL -> RB_3_PL
  VP_3_PL -> V_3_PL N

  RB_1_SG -> "I"
  RB_3_SG -> "she"
  RB_2_SG -> "you"
  RB_3_PL -> "they"
  
  N -> "happy"

  V_1_SG -> "am"
  V_3_SG -> "is"
  V_2_SG -> "are"
  V_3_PL -> "are"

""")

rdp = nltk.RecursiveDescentParser(grammar)

for tree in rdp.parse(sentc_1):
  print(tree)

(S (NP_1_SG (RB_1_SG I)) (VP_1_SG (V_1_SG am) (N happy)))


In [ ]:
grammar2 = nltk.CFG.fromstring("""
  S -> NP[AGR=?n] VP[AGR=?n]
  NP[AGR=?n] = RB[AGR=?n]
  VP[TENSE=?t, AGR=?n] -> V[TENSE=?t, AGR=?n] N

  RB[AGR=[PER=1, NUM=SG]] -> "I"
  RB[AGR=[PER=2, NUM=SG]] -> "you"
  RB[AGR=[PER=3, NUM=SG]] -> "she"
  RB[AGR=[PER=3, NUM=PL]] -> "they"

  V[TENSE = pres, AGR=[PER=1, NUM=SG]] -> "am"
  V[TENSE = pres, AGR=[PER=2, NUM=SG]] -> "are"
  V[TENSE = pres, AGR=[PER=3, NUM=SG]] -> "is"
  V[TENSE = pres, AGR=[PER=3, NUM=PL]] -> "are"

  N -> "happy"

""")


# Exercise 2

☼ Develop a variant of grammar in 1.1 that uses a feature count to make the distinctions shown below:

```
(54)		
a.		The boy sings.
b.		*Boy sings.

(55)		
a.		The boys sing.
b.		Boys sing.

(56)		
a.		The boys sing.
b.		Boys sing.

(57)		
a.		The water is precious.
b.		Water is precious.

```

--- 

Handled all the cases except for the `Boy sings.` being wrong. It is not clear to me if that structure is wrong, and if it is how do I diffferentiate that case from the others. 

In [ ]:
"""
############################
## Grammar Produtions ##
############################

## S Expansion Productions

S -> NP[NUM=?N] VP[NUM=?N]

NP[NUM=?n] -> Det N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl]

VP[NUM=?n] -> IV[NUM=?n]
VP[NUM=?n] -> TV[NUM=?n] N

############################
#### Lexical Produtions ####
############################

Det -> 'the'
N[NUM=sg] -> 'boy' | 'water'
N[NUM=pl] -> 'boys'
N -> 'precious'

IV[NUM=sg] -> 'sings'
IV[NUM=pl] -> 'sing'
TV[NUM=sg] -> 'is'

"""

# Exercise 3

☼ Write a function subsumes() which holds of two feature structures fs1 and fs2 just in case fs1 subsumes fs2.

In [10]:
fs_agr = nltk.FeatStruct(PER=3, NUM = 'pl', GND = 'fem')
fs_tense_pres = nltk.FeatStruct(TENSE = 'pres')
fs_id = nltk.FeatStruct(NAME = 'ANUDEEP')
fs_address = nltk.FeatStruct(HOUSE_NUMBER=79, STREET = "5th MAIN", CITY = "BANGALORE")

fs_combo = nltk.FeatStruct(fs_id, ADDRESS = fs_address)

def subsumes(fs1, fs2):
  if fs1.unify(fs2) == fs2:
    return fs1, fs2
    
  elif fs1.unify(fs2) == fs1:
    return fs2, fs1 
  
  else:
    return fs1, fs2, fs1.unify(fs2)
  

print(subsumes(fs_id, fs_address), end ='\n\n')

print('\n------------------------------------\n')

print(fs_combo)

([NAME='ANUDEEP'], [CITY='BANGALORE', HOUSE_NUMBER=79, STREET='5th MAIN'], [CITY='BANGALORE', HOUSE_NUMBER=79, NAME='ANUDEEP', STREET='5th MAIN'])


------------------------------------

[           [ CITY         = 'BANGALORE' ] ]
[ ADDRESS = [ HOUSE_NUMBER = 79          ] ]
[           [ STREET       = '5th MAIN'  ] ]
[                                          ]
[ NAME    = 'ANUDEEP'                      ]


# Exercise 4. 

☼ Modify the grammar illustrated in (28) to incorporate a bar feature for dealing with phrasal projections.

In [ ]:
"""

VP[TENSE=?t, NUM=?n] -> V[SUBCAT=intrans, TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=trans, TENSE=?t, NUM=?n] NP
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=clause, TENSE=?t, NUM=?n] SBar

V[SUBCAT=intrans, TENSE=pres, NUM=sg] -> 'disappears' | 'walks'
V[SUBCAT=trans, TENSE=pres, NUM=sg] -> 'sees' | 'likes'
V[SUBCAT=clause, TENSE=pres, NUM=sg] -> 'says' | 'claims'

V[SUBCAT=intrans, TENSE=pres, NUM=pl] -> 'disappear' | 'walk'
V[SUBCAT=trans, TENSE=pres, NUM=pl] -> 'see' | 'like'
V[SUBCAT=clause, TENSE=pres, NUM=pl] -> 'say' | 'claim'

V[SUBCAT=intrans, TENSE=past, NUM=?n] -> 'disappeared' | 'walked'
V[SUBCAT=trans, TENSE=past, NUM=?n] -> 'saw' | 'liked'
V[SUBCAT=clause, TENSE=past, NUM=?n] -> 'said' | 'claimed'

"""

In [ ]:
"""

S -> N[BAR=2] V[BAR=2]
N[BAR=2] -> N[BAR=1]
N[BAR=1] -> N[BAR=0] XS

V[BAR=2] -> V[BAR=1]
V[BAR=1] -> V[BAR=1] N[BAR=2]
V[BAR=1] -> V[BAR=1] PP[BAR=2]
V[BAR=1] -> V[BAR=0] XS

VP[TENSE=?t, NUM=?n] -> V[SUBCAT=clause, TENSE=?t, NUM=?n] SBar

SBar/PP[BAR=2] -> P[BAR=1]
P[BAR=1] -> P[BAR=1] S
P[BAR=1] -> P[BAR=0] XS

"""

# Exercise 5

☼ Modify the German grammar in 3.2 to incorporate the treatment of subcategorization presented in 3.

In [ ]:
"""

 # Grammar Productions

 S -> NP[CASE=nom, AGR=?a] VP[AGR=?a]
 NP[CASE=?c, AGR=?a] -> PRO[CASE=?c, AGR=?a]
 NP[CASE=?c, AGR=?a] -> Det[CASE=?c, AGR=?a] N[CASE=?c, AGR=?a]
 VP[AGR=?a] -> V[SUBCAT=IV, AGR=?a]
 VP[AGR=?a] -> V[SUBCAT=TV, OBJCASE=?C, AGR=?A] NP[CASE=?c]

 """

In [19]:
nltk.parse.earleychart.demo(print_grammar=True, trace=4)

"""
1. Initialize an entire leaf table 
2. Start Loop:
  - Start state -> starts with the S production state 
      Sets the dot at pos i=0, with all the production rules to the right
  
  - Predictor -> reads the productions, and finds the rules that start with value immediately to the right of the dot
      looks up all the productions and finds all the rules and adds them to the "processing queue"
  
  - Scanner -> finds the rule in the "processing queue" that ends in a terminal and matches the symbol received, and moves the dot to the right 

  - Completer -> 



"""

* Grammar
Grammar with 18 productions (start state = S)
    S -> NP VP
    PP -> 'with' NP
    NP -> NP PP
    VP -> VP PP
    VP -> Verb NP
    VP -> Verb
    NP -> Det Noun
    NP -> 'John'
    NP -> 'I'
    Det -> 'the'
    Det -> 'my'
    Det -> 'a'
    Noun -> 'dog'
    Noun -> 'cookie'
    Verb -> 'ate'
    Verb -> 'saw'
    Prep -> 'with'
    Prep -> 'under'
* Sentence:
I saw John with a dog with my cookie
['I', 'saw', 'John', 'with', 'a', 'dog', 'with', 'my', 'cookie']

|. I  .saw .John.with. a  .dog .with. my .cook.|
Leaf Init Rule:
|[----]    .    .    .    .    .    .    .    .| [0:1] 'I'
|.    [----]    .    .    .    .    .    .    .| [1:2] 'saw'
|.    .    [----]    .    .    .    .    .    .| [2:3] 'John'
|.    .    .    [----]    .    .    .    .    .| [3:4] 'with'
|.    .    .    .    [----]    .    .    .    .| [4:5] 'a'
|.    .    .    .    .    [----]    .    .    .| [5:6] 'dog'
|.    .    .    .    .    .    [----]    .    .| [6:7] 'with'
|.    .    .    .    .    

'\n1. Initialize an entire leaf table \n2. Start Loop:\n  - Start state -> starts with the S production state \n      Sets the dot at pos i=0, with all the production rules to the right\n  \n  - Predictor -> reads the productions, and finds the rules that start with value immediately to the right of the dot\n      looks up all the productions and finds all the rules and adds them to the "processing queue"\n  \n  - Scanner -> finds the rule in the "processing queue" that ends in a terminal and matches the symbol received, and moves the dot to the right \n\n  - Completer -> \n\n\n\n'

# Exercise 8

Work out on paper what the result is of the following unifications. (Hint: you might find it useful to draw the graph structures.)

- fs1 and fs2
- fs1 and fs3
- fs4 and fs5
- fs5 and fs6
- fs5 and fs7
- fs8 and fs9
- fs8 and fs10

Check your answers using Python.


```
fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs2 = nltk.FeatStruct("[B = [D = d]]")
fs3 = nltk.FeatStruct("[B = [C = d]]")
fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs6 = nltk.FeatStruct("[A = [D = d]]")
fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")
fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")
fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")

```

In [2]:
fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs2 = nltk.FeatStruct("[B = [D = d]]")
fs3 = nltk.FeatStruct("[B = [C = d]]")
fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs6 = nltk.FeatStruct("[A = [D = d]]")
fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")

fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")

fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")


fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")

In [13]:
print(fs1.unify(fs2))
print("\n----------------------\n")
print(fs1.unify(fs3))
print("\n----------------------\n")
print(fs4.unify(fs5))
print("\n----------------------\n")
print(fs6.unify(fs5))
print("\n----------------------\n")
print(fs5.unify(fs7))
print("\n----------------------\n")
print(fs8.unify(fs9))
print("\n----------------------\n")
print(fs8.unify(fs10))

[ A = ?x          ]
[                 ]
[ B = [ C = ?x  ] ]
[     [ D = 'd' ] ]

----------------------

[ A = 'd'         ]
[                 ]
[ B = [ C = 'd' ] ]

----------------------

[         [ B = 'b'  ] ]
[ A = (1) [ D = ?x   ] ]
[         [ E -> (1) ] ]
[         [ F = ?x   ] ]
[                      ]
[ C -> (1)             ]

----------------------

[ A = (1) [ D = 'd' ] ]
[                     ]
[ C = [ E -> (1) ]    ]
[     [ F = 'd'  ]    ]

----------------------

None

----------------------

[         [ B = 'b' ] ]
[ A = (1) [ D = 'e' ] ]
[         [ G = 'e' ] ]
[                     ]
[ C = [ B = 'e'  ]    ]
[     [ E -> (1) ]    ]

----------------------

[         [ B = 'b'  ] ]
[ A = (1) [ D = 'b'  ] ]
[         [ E -> (1) ] ]
[         [ G = 'b'  ] ]
[                      ]
[ C -> (1)             ]
